# DNN 모델을 이용한 Cifar-10 데이터셋 분류

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## GPU 설정

In [ ]:
# 코드작성] GPU 상태 체크
!nvidia-smi

Thu Nov  5 13:11:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    32W /  70W |    235MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## step1. Cifar-10 datasets 준비

In [ ]:
# import 
from tensorflow.keras.datasets import cifar10
import matplotlib as plt

In [ ]:
#  cifar10 데이터셋 load_data()
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [ ]:
#  train, test dataset 탐색 : shape
print(train_images.shape)
print(test_images.shape)
print(test_images[0].shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(32, 32, 3)


### 1-1 분류명을 직관적으로 출력

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

### 1-2 이미지 전처리
- 이미지의 픽셀값을 0과 1 사이로 스케일을 조정 - 정규화
- 먼저 float32 로 typecasting 후 정규화

In [ ]:
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

In [ ]:
# flatten
train_images = train_images.reshape(50000, 32*32*3)
test_images = test_images.reshape(10000, 32*32*3)
print(train_images.shape)
print(test_images.shape)

(50000, 3072)
(10000, 3072)


### 1-3 레이블 인코딩
- 레이블을 범주형으로 인코딩
- to_categorical 사용

In [ ]:
# to_categorical import 
from tensorflow.keras.utils import to_categorical
# 인수로 넘겨받은 리스트에서 유니크한 것만 골라서 카테고리로 사용

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
print(train_labels[0])

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


## step2. DNN 모델 디자인

In [ ]:
# models, layers import 
from tensorflow.keras import models, layers

In [ ]:
# Sequential 모델 객체 생성
model = models.Sequential()
model.add(layers.Dense(units=1024, 
                       activation='relu', 
                       input_shape=(32 * 32 * 3,)))
model.add(layers.Dense(units=512, 
                       activation='relu'))
model.add(layers.Dense(units=256,
                       activation='relu'))
model.add(layers.Dense(units=10, 
                       activation='softmax'))


In [ ]:
# model summary 보기
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 1024)              3146752   
_________________________________________________________________
dense_25 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_26 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_27 (Dense)             (None, 10)                2570      
Total params: 3,805,450
Trainable params: 3,805,450
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 1 hidden layers parameter : 32 * 32 * 3 * 512 + 512
# 2 hidden layers parameter : 512 * 256 + 256


## step3. 훈련 준비하기 (컴파일 단계)
- optimizer : rmsprop
- loss (손실함수 ) : categorical_crossentropy
- metrics : accuracy

In [ ]:
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy',])

In [ ]:
50000 * 0.8 / 64

625.0

## step4. 훈련하기 
- x : 입력 데이터
- y : 입력 라벨 값
- batch_size : 64
- epochs : 30
- validation_split : 0.2

In [ ]:
history = model.fit(train_images, train_labels,
                    epochs=50, 
                    batch_size=64,   # step
                    validation_split=0.2)

Epoch 1/50
625/625 [==============================] - 3s 5ms/step - loss: 2.0792 - accuracy: 0.2721 - val_loss: 1.8111 - val_accuracy: 0.3448
Epoch 2/50
625/625 [==============================] - 3s 4ms/step - loss: 1.7668 - accuracy: 0.3654 - val_loss: 1.7659 - val_accuracy: 0.3733
Epoch 3/50
625/625 [==============================] - 3s 4ms/step - loss: 1.6737 - accuracy: 0.4017 - val_loss: 1.6378 - val_accuracy: 0.4168
Epoch 4/50
625/625 [==============================] - 3s 4ms/step - loss: 1.6136 - accuracy: 0.4248 - val_loss: 1.6921 - val_accuracy: 0.4112
Epoch 5/50
625/625 [==============================] - 3s 4ms/step - loss: 1.5710 - accuracy: 0.4410 - val_loss: 1.6557 - val_accuracy: 0.4098
Epoch 6/50
625/625 [==============================] - 3s 4ms/step - loss: 1.5332 - accuracy: 0.4550 - val_loss: 1.5631 - val_accuracy: 0.4413
Epoch 7/50
625/625 [==============================] - 3s 4ms/step - loss: 1.5076 - accuracy: 0.4643 - val_loss: 1.6102 - val_accuracy: 0.4430
Epoch 

## step5. 모델 평가하기 

In [ ]:
# test_images로 모델 평가하기 
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 2.0551 - accuracy: 0.4410


## Step6. 성능 시각화

In [ ]:
# matplotlib import 


- Training and validation accuracy 그리기

- Training and validation loss 그래프 그리기

In [ ]:
# Training and validation loss 그래프 그리기


## step7. 모델 사용하기

In [ ]:
# numpy import 


In [ ]:
# test_image[0]로 예측하기



In [ ]:
# test_image[0] 이미지보기


In [20]:
!git branch -d main

error: branch 'main' not found.
